# Load and Create Reader

In [1]:
from mtga import base

help(base.GameDataReader)

Help on class GameDataReader in module mtga.base:

class GameDataReader(MTGReader)
 |  GameDataReader(set_code, limited_type, dat_path='~/dat/17Lands', chunk_size=10000)
 |  
 |  Method resolution order:
 |      GameDataReader
 |      MTGReader
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, set_code, limited_type, dat_path='~/dat/17Lands', chunk_size=10000)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  cards_to_cards_sparse(self, cards)
 |  
 |  get_data(self, force_refresh=False)
 |  
 |  set_card_meta(self, header)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from MTGReader:
 |  
 |  setup_disk_meta(self, set_code, limited_type, data_type, dat_path)
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties inherited from MTGReader:
 |  
 |  n_lines
 |  
 |  -------------------------------------------------------------

In [2]:
print(f"Sets: {base.SETS}")
print(f"Limited Formats: {base.LIMITED_TYPES}")
print(f"Data Types: {base.DATA_TYPES}")

Sets: ['LTR']
Limited Formats: ['PremierDraft', 'TradDraft', 'Sealed', 'TradSealed']
Data Types: ['draft', 'game', 'replay']


In [3]:
reader = base.GameDataReader("LTR", "PremierDraft")

2024-07-24 20:02:42,293 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L103 :: Created `GameDataReader with the following non-card columns:
expansion, event_type, draft_id, draft_time, game_time, build_index, match_number, game_number, rank, opp_rank, main_colors, splash_colors, on_play, num_mulligans, opp_num_mulligans, opp_colors, num_turns, won, user_n_games_bucket, user_game_win_rate_bucket


# Generate Sparse Data

In [4]:
data = reader.get_data(force_refresh=True)

2024-07-24 20:02:48,220 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 1/99.
2024-07-24 20:02:49,113 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 2/99.
2024-07-24 20:02:49,970 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 3/99.
2024-07-24 20:02:50,866 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 4/99.
2024-07-24 20:02:51,706 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 5/99.
2024-07-24 20:02:52,551 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 6/99.
2024-07-24 20:02:53,425 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 7/99.
2024-07-24 20:02:54,266 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 8/99.
2024-07-24 20:02:55,087 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed chunk 9/99.
2024-07-24 20:02:55,897 UTC :: /Users/bward92/src/mtga/mtga/base.py :: L171 :: Processed ch

In [5]:
card_data = data["card_data"]
noncard_data = data["noncard_data"]

# Basic Analysis

We replicate some basic metrics defined [here](https://www.17lands.com/metrics_definitions). In particular, below we calculate weighted average win rates where the weight is equal to the card's count. 

In [6]:
import numpy as np
import pandas as pd


# table
wins = noncard_data['won'].values
rates = np.zeros(card_data.shape[1])

# for each (card, position)
for i in range(card_data.shape[1]):

    # carve out (card, position) counts
    s = slice(card_data.indptr[i], card_data.indptr[i+1])
    counts = card_data.data[s]

    # calculate wavg win rate
    rates[i] = (wins[card_data.indices[s]] * counts).sum() / counts.sum()

# create table
index = pd.MultiIndex.from_tuples(zip(reader.card_positions, reader.card_names))
rates = pd.Series(rates, index=index).unstack(0)

# top cards, sorted by position
position = 'deck'
rates.dropna(subset=[position])\
     .sort_values(position, ascending=False)\
     .head(10)

2024-07-24 20:04:09,022 UTC :: /Users/bward92/miniconda3/envs/mtga/lib/python3.10/warnings.py :: L109 :: /var/folders/59/v1qm9t0j4gq0p8vc3049nlm00000gn/T/ipykernel_24428/2752819338.py:17: RuntimeWarning: invalid value encountered in scalar divide
  rates[i] = (wins[card_data.indices[s]] * counts).sum() / counts.sum()



,deck,drawn,opening_hand,sideboard,tutored
Orcish Bowmasters,0.620556,0.702921,0.727410,0.553263,0.711340
"Andúril, Flame of the West",0.607237,0.699681,0.709953,0.463918,0.678161
Horn of Gondor,0.601030,0.683932,0.673046,0.541086,0.727273
Witch-king of Angmar,0.593143,0.665229,0.677845,0.536640,0.698276
Éomer of the Riddermark,0.589089,0.637528,0.625802,0.557386,0.697876
"Fear, Fire, Foes!",0.588480,0.643598,0.611142,0.552025,0.661082
Rally at the Hornburg,0.587808,0.613387,0.614676,0.555458,0.649343
Nazgûl,0.587726,0.633275,0.628526,0.550797,0.666667
"Grishnákh, Brash Instigator",0.585558,0.623329,0.609229,0.548958,0.657807
The Torment of Gollum,0.583167,0.611982,0.602036,0.562124,0.624000
